In [1]:
import datetime
from functools import partial

import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt

from utils import example_to_tensor, train_test_split
from data_augmentation import random_rotate, random_flip
from plot import plot_slice, plot_volume_animation
from config import PATCH_SHAPE

%matplotlib inline
plt.rcParams["figure.figsize"] = [15, 7]

In [34]:
test_perc = 0.1
val_perc = 0.12
epochs = 1000
patience = 30
batch_size = 16
learning_rate = 0.0001
dropout_rate = 0.5

In [3]:
def normalize(volume):
    "Normalize the input volume with values in [0, 1]"
    min_value = tf.reduce_min(volume)
    max_value = tf.reduce_max(volume)
    return (volume - min_value) / (max_value - min_value)

In [4]:
neg_x = (
    tf.data.TFRecordDataset("/pclhcb06/emilio/neg_nodule.tfrecord")
    .map(example_to_tensor, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .map(normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
)
# num_neg_samples = sum(1 for _ in neg_x)
num_neg_samples = 375
print(num_neg_samples)
neg_x

375


<ParallelMapDataset shapes: (None, None, None, None), types: tf.float32>

In [5]:
neg_y = tf.data.Dataset.from_tensor_slices(np.int8([[0]])).repeat(num_neg_samples)
neg_y

<RepeatDataset shapes: (1,), types: tf.int8>

In [6]:
neg_dataset = tf.data.Dataset.zip((neg_x, neg_y))
neg_dataset

<ZipDataset shapes: ((None, None, None, None), (1,)), types: (tf.float32, tf.int8)>

In [7]:
pos_x = (
    tf.data.TFRecordDataset("/pclhcb06/emilio/pos_nodule.tfrecord")
    .map(example_to_tensor, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .map(normalize, num_parallel_calls=tf.data.experimental.AUTOTUNE)
)
# num_pos_samples = sum(1 for _ in pos_x)
num_pos_samples = 379
print(num_pos_samples)
pos_x

379


<ParallelMapDataset shapes: (None, None, None, None), types: tf.float32>

In [25]:
pos_y = tf.data.Dataset.from_tensor_slices(np.int8([[1]])).repeat(num_pos_samples)
pos_y

<RepeatDataset shapes: (1,), types: tf.int8>

In [26]:
pos_dataset = tf.data.Dataset.zip((pos_x, pos_y))
pos_dataset

<ZipDataset shapes: ((None, None, None, None), (1,)), types: (tf.float32, tf.int8)>

In [27]:
@tf.function
def set_shape(volume, label):
    volume.set_shape(PATCH_SHAPE)
    return volume, label

In [28]:
dataset = neg_dataset.concatenate(pos_dataset)
train_val_dataset, test_dataset = train_test_split(
    dataset, test_perc=test_perc, cardinality=(num_neg_samples + num_pos_samples)
)
train_dataset, val_dataset = train_test_split(train_val_dataset, test_perc=val_perc)
test_dataset = test_dataset.batch(1)
val_dataset = (
    val_dataset.batch(batch_size).cache().prefetch(tf.data.experimental.AUTOTUNE)
)
train_dataset = (
    train_dataset.map(random_rotate, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .map(random_flip, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    # set shape because shape inference after tf.numpy_function doesn't work
    .map(set_shape, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    .batch(batch_size)
    .cache()  # must be called before shuffle
    .shuffle(buffer_size=128, reshuffle_each_iteration=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)
train_dataset

<PrefetchDataset shapes: ((None, 16, 64, 64, 1), (None, 1)), types: (tf.float32, tf.int8)>

In [29]:
SeluConv3D = partial(
    keras.layers.Conv3D,
    padding="same",
    activation="selu",
    kernel_initializer="lecun_normal",
    bias_initializer="zeros",
)

In [30]:
SeluDense = partial(
    keras.layers.Dense,
    activation="selu",
    kernel_initializer="lecun_normal",
    bias_initializer="zeros",
)

In [40]:
cnn = keras.Sequential(
    [
        keras.layers.InputLayer(PATCH_SHAPE),
        SeluConv3D(filters=64, kernel_size=3),
        keras.layers.MaxPool3D(2),
        SeluConv3D(filters=128, kernel_size=3),
        keras.layers.MaxPool3D(2),
        SeluConv3D(filters=256, kernel_size=3),
        keras.layers.MaxPool3D(2),
        keras.layers.Flatten(),
        SeluDense(512),
        keras.layers.AlphaDropout(dropout_rate),
        keras.layers.Dense(1, activation="sigmoid"),
    ],
    name="3dcnn",
)
cnn.summary()

Model: "3dcnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_9 (Conv3D)            (None, 16, 64, 64, 64)    1792      
_________________________________________________________________
max_pooling3d_9 (MaxPooling3 (None, 8, 32, 32, 64)     0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 8, 32, 32, 128)    221312    
_________________________________________________________________
max_pooling3d_10 (MaxPooling (None, 4, 16, 16, 128)    0         
_________________________________________________________________
conv3d_11 (Conv3D)           (None, 4, 16, 16, 256)    884992    
_________________________________________________________________
max_pooling3d_11 (MaxPooling (None, 2, 8, 8, 256)      0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 32768)             0     

In [41]:
cnn.compile(
    optimizer=keras.optimizers.Adam(learning_rate),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)

In [ ]:
monitor_metric = "val_accuracy"

start_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
best_checkpoint = f"models/baseline-{start_time}.h5"
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    best_checkpoint, monitor=monitor_metric, verbose=1, save_best_only=True
)
early_stopping_cb = keras.callbacks.EarlyStopping(
    monitor=monitor_metric,
    patience=patience,
)
log_dir = f"logs/baseline-{start_time}"
file_writer = tf.summary.create_file_writer(log_dir)
with file_writer.as_default():
    tf.summary.text(
        "Hyperparameters",
        f"{PATCH_SHAPE=}; "
        f"{epochs=}; "
        f"{test_perc=}; "
        f"{val_perc=}; "
        f"{patience=}; "
        f"{batch_size=}; "
        f"{dropout_rate=}; "
        f"{learning_rate=}; ",
        step=0,
    )
tensorboard_cb = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir,
    histogram_freq=1,
    write_graph=False,
    profile_batch=0,
)
cnn.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs,
    callbacks=[checkpoint_cb, early_stopping_cb, tensorboard_cb],
)
cnn = keras.models.load_model(best_checkpoint)

Epoch 1/1000
     37/Unknown - 2s 45ms/step - loss: 0.8494 - accuracy: 0.5962
Epoch 00001: val_accuracy improved from -inf to 0.70370, saving model to models/baseline-20201112-215903.h5
38/38 [==============================] - 23s 595ms/step - loss: 0.8426 - accuracy: 0.5970 - val_loss: 1.0963 - val_accuracy: 0.7037
Epoch 2/1000
38/38 [==============================] - ETA: 0s - loss: 0.5999 - accuracy: 0.7007
Epoch 00002: val_accuracy improved from 0.70370 to 0.72840, saving model to models/baseline-20201112-215903.h5
38/38 [==============================] - 24s 621ms/step - loss: 0.5999 - accuracy: 0.7007 - val_loss: 0.7864 - val_accuracy: 0.7284
Epoch 3/1000
37/38 [============================>.] - ETA: 0s - loss: 0.5091 - accuracy: 0.7423
Epoch 00003: val_accuracy did not improve from 0.72840
38/38 [==============================] - 3s 82ms/step - loss: 0.5077 - accuracy: 0.7425 - val_loss: 0.7943 - val_accuracy: 0.7160
Epoch 4/1000
37/38 [============================>.] - ETA: 0s 

In [38]:
cnn.evaluate(test_dataset, return_dict=True, verbose=0)

{'loss': 0.9406284689903259, 'accuracy': 0.6933333277702332}

In [39]:
patch, label = next(iter(test_dataset.skip(4)))
print(f"label: {label.numpy()}")
prediction = cnn(patch, training=False)
print(f"prediction: {prediction.numpy()}")
plot_volume_animation(patch[0, :])

label: [[1]]
prediction: [[0.99994457]]
